In [52]:
from __future__ import print_function
import torch
from model import highwayNet
from utils import ngsimDataset, maskedNLL, maskedMSETest, maskedNLLTest
from torch.utils.data import DataLoader
import time

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
#device = torch.device('cpu')
args = {}
args['use_cuda'] = True
args['encoder_size'] = 64
args['decoder_size'] = 128
args['in_length'] = 16
args['out_length'] = 25
args['grid_size'] = (13, 3)
args['soc_conv_depth'] = 64
args['conv_3x1_depth'] = 16
args['dyn_embedding_size'] = 32
args['input_embedding_size'] = 32
args['num_lat_classes'] = 3
args['num_lon_classes'] = 2
args['use_maneuvers'] = False
args['train_flag'] = False

net = highwayNet(args)
net.load_state_dict(torch.load('trained_models/cslstm_m_0.pt'))
net = net.to(device)
tsSet = ngsimDataset(r'C:\Users\Hailong\Projects\ngsim_preprocessed_oldgit\TestSet.mat')
tsDataloader = DataLoader(tsSet, batch_size=128, shuffle=True, num_workers=8, collate_fn=tsSet.collate_fn)

pred = torch.zeros(25,1,2)
gt = torch.zeros(25,1,2)






In [53]:
st_time = time.time()
for i, data in enumerate(tsDataloader):

    hist, nbrs, masks, lat_enc, lon_enc, fut_label, op_mask  = data

    masks = torch.zeros([hist.shape[1], 3, 13, 64], device="cpu").bool()
    # Initialize Variables

    if args['use_cuda']:
        hist = hist.cuda()
        nbrs = nbrs.cuda()
        masks = masks.cuda()
        lat_enc = lat_enc.cuda()
        lon_enc = lon_enc.cuda()
        fut_label = fut_label.cuda()
        op_mask = op_mask.cuda()
    # hist.to(device)
    # nbrs.to(device)
    # # mask = mask.to(device)
    # #op_mask = torch.ones(25,1,2)
    # fut_label.to(device)

    #masks = torch.ones([hist.shape[1], 3, 13, 64], device="cuda").bool()

    # Forward pass
    st_time = time.time()
    fut_pred = net(hist, nbrs, masks)
    time_taken = time.time() - st_time
    print(time_taken)
    pred = fut_pred[:,2,:2]
    gt = fut_label[:,2,:]

    break

0.0050048828125


In [23]:
gt

tensor([[ 2.1700e-01,  4.8750e+00],
        [ 3.0100e-01,  1.0189e+01],
        [ 4.0800e-01,  1.6096e+01],
        [ 3.8100e-01,  2.2442e+01],
        [ 3.7900e-01,  2.8789e+01],
        [ 1.7900e-01,  3.4965e+01],
        [-1.5900e-01,  4.1161e+01],
        [-2.7400e-01,  4.7886e+01],
        [-3.2000e-01,  5.5112e+01],
        [-4.0300e-01,  6.2399e+01],
        [-4.8700e-01,  6.9663e+01],
        [-5.7100e-01,  7.6930e+01],
        [-6.5400e-01,  8.4196e+01],
        [-7.3800e-01,  9.1525e+01],
        [-8.3500e-01,  9.8310e+01],
        [-7.4600e-01,  1.0483e+02],
        [-6.5600e-01,  1.1201e+02],
        [-5.6800e-01,  1.1928e+02],
        [-4.7900e-01,  1.2654e+02],
        [-3.9000e-01,  1.3381e+02],
        [-3.0000e-01,  1.4106e+02],
        [-2.4700e-01,  1.4836e+02],
        [-1.7700e-01,  1.5603e+02],
        [-3.7998e-02,  1.6403e+02],
        [ 1.4000e-01,  1.7204e+02]])

In [31]:


[(gt[i,0] - pred[i,0],gt[i,1] - pred[i,1]) for i in range(0,pred.shape[0]-1)]

[(tensor(-0.2460, grad_fn=<SubBackward0>),
  tensor(0.0613, grad_fn=<SubBackward0>)),
 (tensor(-0.5024, grad_fn=<SubBackward0>),
  tensor(0.1434, grad_fn=<SubBackward0>)),
 (tensor(-0.3526, grad_fn=<SubBackward0>),
  tensor(0.2280, grad_fn=<SubBackward0>)),
 (tensor(-0.2097, grad_fn=<SubBackward0>),
  tensor(0.6160, grad_fn=<SubBackward0>)),
 (tensor(-0.0707, grad_fn=<SubBackward0>),
  tensor(0.7322, grad_fn=<SubBackward0>)),
 (tensor(0.0003, grad_fn=<SubBackward0>),
  tensor(0.5756, grad_fn=<SubBackward0>)),
 (tensor(-0.3072, grad_fn=<SubBackward0>),
  tensor(0.9235, grad_fn=<SubBackward0>)),
 (tensor(-0.1715, grad_fn=<SubBackward0>),
  tensor(1.1191, grad_fn=<SubBackward0>)),
 (tensor(0.5505, grad_fn=<SubBackward0>),
  tensor(1.1087, grad_fn=<SubBackward0>)),
 (tensor(0.7802, grad_fn=<SubBackward0>),
  tensor(0.5268, grad_fn=<SubBackward0>)),
 (tensor(0.7439, grad_fn=<SubBackward0>),
  tensor(-0.0278, grad_fn=<SubBackward0>)),
 (tensor(0.7937, grad_fn=<SubBackward0>),
  tensor(-0.229

In [51]:
[(gt[i,0] - pred[i,0],gt[i,1] - pred[i,1]) for i in range(0,pred.shape[0]-1)]

[(tensor(0.0086, device='cuda:0', grad_fn=<SubBackward0>),
  tensor(0.4272, device='cuda:0', grad_fn=<SubBackward0>)),
 (tensor(-0.0103, device='cuda:0', grad_fn=<SubBackward0>),
  tensor(1.1993, device='cuda:0', grad_fn=<SubBackward0>)),
 (tensor(-0.0167, device='cuda:0', grad_fn=<SubBackward0>),
  tensor(0.4899, device='cuda:0', grad_fn=<SubBackward0>)),
 (tensor(-0.0227, device='cuda:0', grad_fn=<SubBackward0>),
  tensor(0.6104, device='cuda:0', grad_fn=<SubBackward0>)),
 (tensor(0.0048, device='cuda:0', grad_fn=<SubBackward0>),
  tensor(1.3968, device='cuda:0', grad_fn=<SubBackward0>)),
 (tensor(0.0004, device='cuda:0', grad_fn=<SubBackward0>),
  tensor(2.2555, device='cuda:0', grad_fn=<SubBackward0>)),
 (tensor(-0.0085, device='cuda:0', grad_fn=<SubBackward0>),
  tensor(2.2133, device='cuda:0', grad_fn=<SubBackward0>)),
 (tensor(-0.0310, device='cuda:0', grad_fn=<SubBackward0>),
  tensor(2.2780, device='cuda:0', grad_fn=<SubBackward0>)),
 (tensor(-0.0455, device='cuda:0', grad_fn=

In [54]:
[(gt[i,0] - pred[i,0],gt[i,1] - pred[i,1]) for i in range(0,pred.shape[0]-1)]

[(tensor(0.1543, device='cuda:0', grad_fn=<SubBackward0>),
  tensor(-0.1117, device='cuda:0', grad_fn=<SubBackward0>)),
 (tensor(0.4375, device='cuda:0', grad_fn=<SubBackward0>),
  tensor(-0.5461, device='cuda:0', grad_fn=<SubBackward0>)),
 (tensor(0.9964, device='cuda:0', grad_fn=<SubBackward0>),
  tensor(-0.7245, device='cuda:0', grad_fn=<SubBackward0>)),
 (tensor(1.5082, device='cuda:0', grad_fn=<SubBackward0>),
  tensor(-0.8277, device='cuda:0', grad_fn=<SubBackward0>)),
 (tensor(2.0265, device='cuda:0', grad_fn=<SubBackward0>),
  tensor(-0.5812, device='cuda:0', grad_fn=<SubBackward0>)),
 (tensor(2.4533, device='cuda:0', grad_fn=<SubBackward0>),
  tensor(-0.4450, device='cuda:0', grad_fn=<SubBackward0>)),
 (tensor(2.6490, device='cuda:0', grad_fn=<SubBackward0>),
  tensor(-0.5330, device='cuda:0', grad_fn=<SubBackward0>)),
 (tensor(2.6016, device='cuda:0', grad_fn=<SubBackward0>),
  tensor(-0.5945, device='cuda:0', grad_fn=<SubBackward0>)),
 (tensor(2.4571, device='cuda:0', grad_f

In [ ]:
#b,c,t,n
model = highwayNet(args)
tracks = torch.randn([2,16,120])

#assumes tracks is c,t,n, c=4, first two in c is velocity
def traj_pred(model, tracks):
    #for each vehicle

    tracks = tracks.reshape(1, tracks.shape[0], tracks.shape[1], tracks.shape[2]) # c t n -> b c t n
    predictions = torch.zeros(2, 25, tracks.shape[3])
    for i in range(tracks.shape[3]):
        # proj = tracks[:,:,-1,i].repeat(tracks.shape[3], 1)
        # assert proj.shape == tracks.shape, "proj and tracks mismatch"
        transform = tracks[:,:,-1,i]
        transformed_tracks = tracks - transform # transforms the coordinate frame to target vehicle at t=0

        nbrs = transformed_tracks.reshape(transformed_tracks.shape[1],transformed_tracks.shape[2], transformed_tracks.shape[3])
        nbrs = nbrs.permute(1,2,0) #c,t,n -> t,n,c
        assert nbrs.shape == (tracks.shape[2], tracks.shape[3], tracks.shape[1]), "nbrs shape error"

        hist = transformed_tracks[:,:,:,i].reshape(transformed_tracks.shape[0],transformed_tracks.shape[1], transformed_tracks.shape[2])
        hist = hist.permute(2,0,1) #b,c,t -> t, b, c
        assert hist.shape == (tracks.shape[2], tracks.shape[0], tracks.shape[1]), "hist shape error"

        masks = torch.zeros([hist.shape[1], 3, 13, 64], device="cuda" if torch.cuda.is_available() else 'cpu').bool()
        fut = model(hist, nbrs, masks)
        fut = fut_pred[:,:,:2]

        fut = fut.reshape(1,fut.shape[0], fut.shape[1], fut.shape[2]) #t b c -> 1 t b c
        fut = fut.permute(2,3,1,0) #n t b c -> b c t n
        fut = fut + transform
        fut = fut.reshape(fut.shape[1], fut.shape[2], fut.shape[3])

        predictions[:,:,i] = fut

        return predictions


traj_pred(model, tracks)



